models

In [ ]:
package models

type User struct {
    ID       uint   `gorm:"primaryKey"`
    Username string `gorm:"unique"`
    Password string
}

type Book struct {
    ID          uint   `gorm:"primaryKey"`
    Title       string
    Author      string
    Description string
    Available   bool
}

type Library struct {
    ID      uint   `gorm:"primaryKey"`
    Name    string
    Address string
    Books   []Book
}


Define Routes and Handlers:

In [ ]:
package main

import (
    "net/http"

    "github.com/gin-gonic/gin"
    "gorm.io/driver/sqlite"
    "gorm.io/gorm"
)

func main() {
    db, err := gorm.Open(sqlite.Open("library.db"), &gorm.Config{})
    if err != nil {
        panic("failed to connect database")
    }

    // Migrate the schema
    db.AutoMigrate(&models.User{}, &models.Book{}, &models.Library{})

    r := gin.Default()

    // Routes
    r.POST("/users", createUser)
    r.POST("/login", login)

    admin := r.Group("/admin")
    admin.Use(authMiddleware)
    {
        admin.POST("/libraries", createLibrary)
        admin.POST("/books", addBook)
        admin.PUT("/books/:id", updateBook)
        admin.DELETE("/books/:id", removeBook)
    }

    user := r.Group("/user")
    user.Use(authMiddleware)
    {
        user.GET("/books", listBooks)
        user.POST("/books/:id/request", requestBook)
    }

    r.Run(":8080")
}

func authMiddleware(c *gin.Context) {
    // Your authentication logic here
    // For simplicity, you can use JWT or session-based authentication
    // Check if the user is authenticated and assign role (admin/user)
}

func createUser(c *gin.Context) {
    // Implementation to create a new user
}

func login(c *gin.Context) {
    // Implementation for user login
}

func createLibrary(c *gin.Context) {
    // Implementation to create a new library (admin only)
}

func addBook(c *gin.Context) {
    // Implementation to add a book to a library (admin only)
}

func updateBook(c *gin.Context) {
    // Implementation to update book details (admin only)
}

func removeBook(c *gin.Context) {
    // Implementation to remove a book from a library (admin only)
}

func listBooks(c *gin.Context) {
    // Implementation to list available books (user only)
}

func requestBook(c *gin.Context) {
    // Implementation to request a book (user only)
}


In [ ]:
package main

import (
    "net/http"

    "github.com/gin-gonic/gin"
    "gorm.io/driver/sqlite"
    "gorm.io/gorm"
)

var db *gorm.DB

type User struct {
    ID       uint   `gorm:"primaryKey"`
    Username string `gorm:"unique"`
    Password string
}

type Book struct {
    ID          uint   `gorm:"primaryKey"`
    Title       string
    Author      string
    Description string
    Available   bool
}

type Library struct {
    ID      uint   `gorm:"primaryKey"`
    Name    string
    Address string
    Books   []Book
}

func main() {
    // Initialize the database
    initDB()

    r := gin.Default()

    // Routes
    r.POST("/users", createUser)
    r.POST("/login", login)

    admin := r.Group("/admin")
    admin.Use(authMiddleware)
    {
        admin.POST("/libraries", createLibrary)
        admin.POST("/books", addBook)
        admin.PUT("/books/:id", updateBook)
        admin.DELETE("/books/:id", removeBook)
    }

    user := r.Group("/user")
    user.Use(authMiddleware)
    {
        user.GET("/books", listBooks)
        user.POST("/books/:id/request", requestBook)
    }

    r.Run(":8080")
}

func initDB() {
    var err error
    db, err = gorm.Open(sqlite.Open("library.db"), &gorm.Config{})
    if err != nil {
        panic("failed to connect database")
    }

    // Migrate the schema
    db.AutoMigrate(&User{}, &Book{}, &Library{})
}

func authMiddleware(c *gin.Context) {
    // Your authentication logic here
}

func createUser(c *gin.Context) {
    var user User
    c.BindJSON(&user)

    // Save user to database
    db.Create(&user)

    c.JSON(http.StatusCreated, gin.H{"message": "User created successfully"})
}

func login(c *gin.Context) {
    // Implementation for user login
}

func createLibrary(c *gin.Context) {
    var library Library
    c.BindJSON(&library)

    // Save library to database
    db.Create(&library)

    c.JSON(http.StatusCreated, gin.H{"message": "Library created successfully"})
}

func addBook(c *gin.Context) {
    var book Book
    c.BindJSON(&book)

    // Save book to database
    db.Create(&book)

    c.JSON(http.StatusCreated, gin.H{"message": "Book added successfully"})
}

func updateBook(c *gin.Context) {
    id := c.Param("id")
    var book Book
    db.First(&book, id)
    c.BindJSON(&book)
    db.Save(&book)

    c.JSON(http.StatusOK, gin.H{"message": "Book updated successfully"})
}

func removeBook(c *gin.Context) {
    id := c.Param("id")
    var book Book
    db.Delete(&book, id)

    c.JSON(http.StatusOK, gin.H{"message": "Book deleted successfully"})
}

func listBooks(c *gin.Context) {
    var books []Book
    db.Find(&books)

    c.JSON(http.StatusOK, books)
}

func requestBook(c *gin.Context) {
    // Implementation to request a book (user only)
}


In [ ]:
func requestBook(c *gin.Context) {
    // Get book ID from URL parameters
    id := c.Param("id")

    // Fetch the book from the database
    var book Book
    if err := db.First(&book, id).Error; err != nil {
        c.JSON(http.StatusNotFound, gin.H{"error": "Book not found"})
        return
    }

    // Check if the book is available
    if !book.Available {
        c.JSON(http.StatusForbidden, gin.H{"error": "Book is not available for request"})
        return
    }

    // Perform request logic (For simplicity, just set book as unavailable)
    book.Available = false
    db.Save(&book)

    c.JSON(http.StatusOK, gin.H{"message": "Book requested successfully"})
}


Sure, let's update the removeBook function to handle deleting books only when the number of copies becomes zero. We'll also need to adjust the Book model to include a field for the number of copies. Here's how you can modify the code:

First, update the Book model to include a field for the number of copies:

In [ ]:
type Book struct {
    ID          uint   `gorm:"primaryKey"`
    Title       string
    Author      string
    Description string
    Available   bool
    Copies      int // Number of copies of the book
}


In [ ]:
func removeBook(c *gin.Context) {
    id := c.Param("id")

    var book Book
    if err := db.First(&book, id).Error; err != nil {
        c.JSON(http.StatusNotFound, gin.H{"error": "Book not found"})
        return
    }

    // Decrease the number of copies by 1
    book.Copies--
    db.Save(&book)

    // If the number of copies is zero, delete the book
    if book.Copies == 0 {
        if err := db.Delete(&book).Error; err != nil {
            c.JSON(http.StatusInternalServerError, gin.H{"error": "Failed to delete book"})
            return
        }
    }

    c.JSON(http.StatusOK, gin.H{"message": "Book deleted successfully"})
}


In [ ]:
package main

import (
    "net/http"

    "github.com/gin-gonic/gin"
    "gorm.io/driver/sqlite"
    "gorm.io/gorm"
)

var db *gorm.DB

type User struct {
    ID       uint   `gorm:"primaryKey"`
    Username string `gorm:"unique"`
    Password string
    Role     string // Role of the user (admin or user)
}

type Book struct {
    ID          uint   `gorm:"primaryKey"`
    Title       string
    Author      string
    Description string
    Available   bool
    Copies      int // Number of copies of the book
}

type Library struct {
    ID      uint   `gorm:"primaryKey"`
    Name    string
    Address string
    Books   []Book
}

func main() {
    // Initialize the database
    initDB()

    r := gin.Default()

    // Routes
    r.POST("/users", createUser)
    r.POST("/login", login)

    admin := r.Group("/admin")
    admin.Use(authMiddleware)
    {
        admin.POST("/libraries", createLibrary)
        admin.POST("/books", addBook)
        admin.PUT("/books/:id", updateBook)
        admin.DELETE("/books/:id", removeBook)
        admin.POST("/books/:id/accept", acceptBookRequest)
    }

    user := r.Group("/user")
    user.Use(authMiddleware)
    {
        user.GET("/books", listBooks)
        user.POST("/books/:id/request", requestBook)
    }

    r.Run(":8080")
}

func initDB() {
    var err error
    db, err = gorm.Open(sqlite.Open("library.db"), &gorm.Config{})
    if err != nil {
        panic("failed to connect database")
    }

    // Migrate the schema
    db.AutoMigrate(&User{}, &Book{}, &Library{})
}

func authMiddleware(c *gin.Context) {
    // Your authentication logic here
    // For simplicity, you can use a JWT or session-based authentication

    // Assuming you have a middleware that extracts user information from the request context
    user, ok := c.Get("user")
    if !ok {
        c.JSON(http.StatusUnauthorized, gin.H{"error": "Unauthorized"})
        c.Abort()
        return
    }

    // Check the user's role
    role, ok := user.(User).Role
    if !ok {
        c.JSON(http.StatusUnauthorized, gin.H{"error": "Unauthorized"})
        c.Abort()
        return
    }

    // Set role in the request context
    c.Set("role", role)

    c.Next()
}

func createUser(c *gin.Context) {
    var user User
    c.BindJSON(&user)

    // Save user to database
    db.Create(&user)

    c.JSON(http.StatusCreated, gin.H{"message": "User created successfully"})
}

func login(c *gin.Context) {
    // Implementation for user login
}

func createLibrary(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "admin" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    var library Library
    c.BindJSON(&library)

    // Save library to database
    db.Create(&library)

    c.JSON(http.StatusCreated, gin.H{"message": "Library created successfully"})
}

func addBook(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "admin" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    var book Book
    c.BindJSON(&book)

    // Save book to database
    db.Create(&book)

    c.JSON(http.StatusCreated, gin.H{"message": "Book added successfully"})
}

func updateBook(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "admin" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    id := c.Param("id")
    var book Book
    db.First(&book, id)
    c.BindJSON(&book)
    db.Save(&book)

    c.JSON(http.StatusOK, gin.H{"message": "Book updated successfully"})
}

func removeBook(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "admin" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    id := c.Param("id")
    var book Book
    db.Delete(&book, id)

    c.JSON(http.StatusOK, gin.H{"message": "Book deleted successfully"})
}

func listBooks(c *gin.Context) {
    var books []Book
    db.Find(&books)

    c.JSON(http.StatusOK, books)
}

func requestBook(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "user" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    // Get book ID from URL parameters
    id := c.Param("id")

    // Fetch the book from the database
    var book Book
    if err := db.First(&book, id).Error; err != nil {
        c.JSON(http.StatusNotFound, gin.H{"error": "Book not found"})
        return
    }

    // Check if the book is available
    if !book.Available {
        c.JSON(http.StatusForbidden, gin.H{"error": "Book is not available for request"})
        return
    }

    // Perform request logic (For simplicity, just set book as unavailable)
    book.Available = false
    db.Save(&book)

    c.JSON(http.StatusOK, gin.H{"message": "Book requested successfully"})
}

func acceptBookRequest(c *gin.Context) {
    role, _ := c.Get("role")
    if role != "admin" {
        c.JSON(http.StatusForbidden, gin.H{"error": "Forbidden"})
        return
    }

    // Get book ID from URL parameters
    id := c.Param("id")

    // Fetch the book from the database
    var book Book
    if err := db.First(&book, id).Error; err != nil {
        c.JSON(http.StatusNotFound, gin.H{"error": "Book not found"})
        return
    }

    // Perform acceptance logic (For simplicity, just set book as available)
    book.Available = true
    db.Save(&book)

    c.JSON(http.StatusOK, gin.H{"message": "Book request accepted successfully"})
}
